In [1]:
import os
import sys
from itertools import permutations
import xlrd
import numpy as np
import pandas as pd

In [71]:
#计算两个向量长度
def len_v(vector1,vector2):
    return (np.linalg.norm(vector2),np.linalg.norm(vector1))

#计算两个向量的余弦相似度
def cos(vector1,vector2):
    vector1 = np.array(vector1)
    vector2 = np.array(vector2)
    return np.dot(vector1,vector2)/(np.linalg.norm(vector1)*np.linalg.norm(vector2))

#得到三个相似度
#a[2]是score
#a[2]两个向量长度组成的向量
def rate(a,b):
    sc1 = cos(a[0],b[0])
    sc2 = cos(a[1],b[1])
    sc3 = cos(a[3],b[3])
    return sc1,sc2,sc3

#三个相似度根据权重算出总的相似度
#theta = 0 只用长度
#theta = 1 
def conf(a,b, theta= 0.8):
    thre = 0.7
    if b>thre:
        return ((a[0]+a[1])*theta/2 +(1-theta)*a[2])
    else:
        return ((a[0]+a[1])*theta/2 +(1-theta)*a[2])*(-1)
    
    
def saveResult(res, savePath, videoName):
    if not os.path.exists(savePath):
          os.makedirs(savePath)
    fullPath = os.path.join(savePath, videoName + ".txt")
    print(fullPath)
    with open(fullPath,'w') as resultTxt:
        for i in range(len(res)):
            resultTxt.write(str(i)+" :"+str(res[i][1]))
            resultTxt.write("\n")

In [47]:
def getResult(dpFilePath, theta = 0.8):   
    videoPath = str(dpFilePath).rsplit("/",1)[0]
    videoName = videoPath.split("/")[-1]
    data = pd.read_csv(dpFilePath)
    x = data["X"].values
    y = data["Y"].values
    score = data["score"].values
    if len(x) < 3:
        print("Wrong Trajectory!")
        return None
    

    #从中间开始求相似度
    mid = int(len(x)/2)
    ub = []
    uf = []
    #向两边扩展
    for i in range(mid,0,-1):
        ub.append([(x[i]-x[i+1],y[i]-y[i+1]),(x[i-1]-x[i],y[i-1]-y[i]),score[i]])
    for i in range(mid,len(x)-1):
        uf.append([(x[i]-x[i-1],y[i]-y[i-1]),(x[i+1]-x[i],y[i+1]-y[i]),score[i+1]])
#     print("ub: {}\nuf: {}".format(ub, uf))

    #标准密码向量生成
    #0-8选三个点
    p3 = list(permutations([0,1,2,3,4,5,6,7,8],3))

    m = [(-1,1),(0,1),(1,1),
        (-1,0),(0,0),(1,0),
        (-1,-1),(0,-1),(1,-1)]

    #找任意三个点对应的向量组合
    vectors = []
    for p in p3:
        vectors.append([(m[p[1]][0]-m[p[0]][0],m[p[1]][1]-m[p[0]][1]),(m[p[2]][0]-m[p[1]][0],m[p[2]][1]-m[p[1]][1]),tuple([a+1 for a in p])])

    #把延伸的每三个点组成一个向量
    #加上长度向量
    for i in range(len(ub)):
        ub[i].append(len_v(ub[i][0],ub[i][1]))
    #加上长度向量
    for i in range(len(uf)):
        uf[i].append(len_v(uf[i][0],uf[i][1]))
    #标准向量加上长度向量
    for i in range(len(vectors)):
        vectors[i].append(len_v(vectors[i][0],vectors[i][1]))
    print(ub)
    #综合概率的阈值
    Threshold = 0.6

    #每三个点可能结果存起来
    rb = []
    for i in range(len(ub)):
        temp = []
        for v in vectors:
            ra = rate(ub[i],v)
            if (np.array(ra)>Threshold).all():
                temp.append([conf(ra,ub[i][2], theta),v[2]])
        temp.sort(reverse = True)
        rb.append(temp)
    rf = []
    for i in range(len(uf)):
        temp = []
        for v in vectors:
            ra = rate(uf[i],v)
            if (np.array(ra)>Threshold).all():
                temp.append([conf(ra,uf[i][2], theta),v[2]])
        temp.sort(reverse = True)
        rf.append(temp)

    # print("rb: {}".format(rb))    
    # print("rf: {}".format(rf))
    #拼接，最终结果    
    res = []
    start = (rb.pop(0),rf.pop(0))
    for b in start[0]:
        for f in start[1]:
            if f[1] == tuple(reversed(b[1])):
                res.append([f[0],list(f[1]),0,0])
    # print("res: {}".format(res))            
    #向两边延伸拼接
    num_r = 0
    while rf and rb:
        nb = rb.pop(0)
        nf = rf.pop(0)
        for b in nb:
            for r in res:
                if r[2] == num_r:
                    if b[1][0] == r[1][1] and b[1][1] == r[1][0] and b[1][2] not in r[1]: 
                        tu = r[1].copy()
                        tu.insert(0,b[1][2])
                        res.append([r[0]+b[0],tu,r[2]+1,r[3]])
        for f in nf:
            for r in res:
                if r[3] == num_r:
                    if f[1][0] == r[1][-2] and f[1][1] == r[1][-1] and f[1][2] not in r[1]:
                        tu = r[1].copy()
                        tu.append(f[1][2])
                        res.append([r[0]+f[0],tu,r[2],r[3]+1])
        num_r += 1

    if rb:
        nb = rb.pop(0)
        for b in nb:
            for r in res:
                if r[2] == num_r:
                    if b[1][0] == r[1][1] and b[1][1] == r[1][0] and b[1][2] not in r[1]:
                        tu = r[1].copy()
                        tu.insert(0,b[1][2])
                        res.append([r[0]+b[0],tu,r[2]+1,r[3]])
    #根据置信度排序结果
    res.sort(reverse = True)
    return res

In [101]:
def reconstruct(dpFilePath, savePath):
    videoPath = str(dpFilePath).rsplit("/",1)[0]
    videoName = videoPath.split("/")[-1]
    resDistance = np.array(getResult(dpFilePath, theta = 0))
    resAngle = np.array(getResult(dpFilePath, theta = 1))
    if(resAngle.any() == None or resDistance.any() == None):
        print("no result")
        return None
    i = 0
    res = []
    #删除小于4点的结果
    while i < (len(resAngle)):
        if(len(resAngle[i][1]) < 4):
            resAngle = np.delete(resAngle, i, axis = 0)
            continue
        i = i + 1
    while i < (len(resDistance)):
        if(len(resDistance[i][1]) < 4):
            resDistance = np.delete(resDistance, i, axis = 0)
            continue
        i = i + 1
    #在Distance 结果中查询Angle中的结果
#     resDistanceLen = len(resDistance) if len(resDistance) < 1000 else 1000
        
#     for i in range(resDistanceLen):
#         for j in range(len(resAngle)):
#             if (resDistance[i][1] == resAngle[j][1]):
#                 res.append([resDistance[i], j])
     #在Angle结果中查询Distance中的结果
    resAngleLen = len(resAngle) if len(resAngle) < 1000 else 1000
        
    for i in range(resAngleLen):
        for j in range(len(resDistance)):
            if (resAngle[i][1] == resDistance[j][1]):
                res.append([resAngle[i], j])
    res = np.array(res)
    if (len(res) != 0):
        res = res[np.argsort(res[:,-1])]
        res = res[:, 0]
        print(len(res))
    print(savePath)
    if not os.path.exists(savePath):
        os.makedirs(savePath)
        print("make dir: "+ savePath)
    
    saveResult(res, savePath, videoName)
    res_len = 20 if len(res) > 20 else len(res)
    for i in range(res_len):
        print(res[i])
    return (res)

In [102]:
# 打开标记成功的文件
# 1 成功， 0 失败
workbook = xlrd.open_workbook('match.xlsx')

# 查看工作表
print("sheets：" + str(workbook.sheet_names()))
# 通过文件名获得工作表,获取工作表1
table = workbook.sheet_by_name('无遮挡')
myDict = {}
for i in range(1, len(table.col_values(0))):
    myDict[table.col_values(0)[i]] = table.col_values(2)[i]
# for keys,values in myDict.items():
#     print(keys, values)

sheets：['无遮挡', '遮挡']


In [105]:
resultPath = "results/no-end"
savePath = "/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results"
for roots, dirs, name in os.walk(resultPath):
    dirs.sort()
    for videoName in dirs:
        if(myDict.get(videoName) != 1):
            continue
#         if(videoName != "SSSM-B3-00088"):
#             continue
        print(videoName)
        noEndPath = os.path.join(resultPath, videoName, "opt_tra.csv")
        reconstruct(noEndPath, savePath)

SSSM-000002
[[(-25, -6), (23, 14), 1, (26.92582403567252, 25.709920264364882)], [(23, 14), (-24, -3), 1, (24.186773244895647, 26.92582403567252)]]
[[(-25, -6), (23, 14), 1, (26.92582403567252, 25.709920264364882)], [(23, 14), (-24, -3), 1, (24.186773244895647, 26.92582403567252)]]
173
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results/SSSM-000002.txt
[1.8144846645392492 list([4, 6, 7, 2]) 1 0]
[1.8753657395544587 list([1, 3, 4, 9]) 1 0]
[1.978878347591972 list([4, 6, 7, 9]) 1 0]
[1.978878347591972 list([1, 3, 4, 6]) 1 0]
[1.7343708733513372 list([1, 6, 7, 2]) 1 0]
[1.89876455640406 list([1, 6, 7, 9]) 1 0]
[1.9302113700750674 list([4, 3, 7, 6]) 1 0]
[1.7649627378909274 list([4, 3, 7, 2]) 1 0]
[1.7083747653586285 list([7, 6, 4, 9]) 1 0]
[1.8127423225275592 list([7, 6, 4, 3]) 1 0]
[1.7572723029050363 list([4, 9, 7, 6]) 1 0]
[1.5920236707208963 list([4, 9, 7, 2]) 1 0]
[1.7083747653586285 list([4, 3, 1, 6]) 1 0

[[(-5, -17), (-7, 10), 1, (12.206555615733702, 17.72004514666935)], [(-7, 10), (7, 7), 1, (9.899494936611665, 12.206555615733702)]]
95
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results/SSSM-000098.txt
[1.818019700196846 list([6, 5, 9, 3]) 1 0]
[1.818019700196846 list([5, 4, 8, 2]) 1 0]
[1.818019700196846 list([2, 5, 9, 3]) 1 0]
[1.818019700196846 list([1, 4, 8, 2]) 1 0]
[1.7042839366268856 list([6, 5, 9, 2]) 1 0]
[1.8304725529081527 list([5, 4, 8, 3]) 1 0]
[1.7042839366268856 list([5, 4, 8, 1]) 1 0]
[1.8304725529081527 list([1, 4, 8, 3]) 1 0]
[1.6117330975695063 list([2, 5, 8, 6]) 1 0]
[1.6117330975695063 list([1, 4, 7, 5]) 1 0]
[1.8507305460336116 list([3, 5, 9, 2]) 1 0]
[1.976919162314879 list([2, 4, 8, 3]) 1 0]
[1.8507305460336116 list([2, 4, 8, 1]) 1 0]
[1.5134945750717357 list([3, 1, 7, 6]) 1 0]
[1.6649209146092563 list([3, 1, 7, 2]) 1 0]
[1.6524680618979497 list([6, 5, 8, 2]) 1 0]
[1.652468061897949

[[(55, 22), (-48, 2), 1, (48.041648597857254, 59.23681287847955)], [(-48, 2), (-1, -12), 1, (12.041594578792296, 48.041648597857254)]]
762
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results/SSSM-000140.txt
[2.907088170951782 list([4, 1, 6, 7, 9]) 1 1]
[2.6617697910447116 list([7, 4, 6, 1, 3]) 1 1]
[2.8637645777328076 list([4, 1, 3, 7, 6]) 1 1]
[2.7489972079254064 list([7, 4, 3, 1, 6]) 1 1]
[2.9165509822328497 list([4, 1, 3, 7, 9]) 1 1]
[2.7606415615450555 list([4, 1, 6, 7, 3]) 1 1]
[2.6553370030187224 list([7, 4, 3, 1, 5]) 1 1]
[2.4742696308171226 list([7, 4, 3, 8, 6]) 1 1]
[2.7606415615450555 list([4, 1, 6, 7, 5]) 1 1]
[2.515323181637985 list([7, 4, 6, 1, 9]) 1 1]
[2.515323181637985 list([7, 4, 6, 1, 5]) 1 1]
[2.598368661491725 list([4, 1, 3, 8, 6]) 1 1]
[2.6098941772694864 list([4, 1, 9, 7, 6]) 1 1]
[2.223265971324836 list([5, 1, 9, 4, 6]) 1 1]
[2.170479566824794 list([5, 1, 9, 4, 3]) 1 1]
[2.51623397236

747
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results/SSSM-000176.txt
[3.4707073405722104 list([6, 4, 7, 3, 9, 8]) 2 1]
[3.6595901566019733 list([6, 1, 4, 3, 9, 8]) 2 1]
[3.8117413406387386 list([6, 1, 4, 3, 8, 7]) 2 1]
[3.739787776040651 list([9, 1, 4, 3, 8, 7]) 2 1]
[3.5876365920038857 list([5, 1, 4, 3, 9, 8]) 2 1]
[3.739787776040651 list([5, 1, 4, 3, 8, 7]) 2 1]
[3.948592449909442 list([3, 1, 4, 6, 8, 7]) 2 1]
[3.4270240048125675 list([6, 4, 7, 3, 9, 5]) 2 1]
[3.8598431947225995 list([9, 1, 4, 6, 8, 7]) 2 1]
[3.4707073405722104 list([5, 4, 7, 3, 9, 8]) 2 1]
[3.8598431947225995 list([5, 1, 4, 6, 8, 7]) 2 1]
[3.4539116499834557 list([6, 1, 7, 3, 9, 8]) 2 1]
[3.3382747496257252 list([8, 4, 7, 3, 9, 5]) 2 1]
[3.6159068208423304 list([6, 1, 4, 3, 9, 5]) 2 1]
[3.4904259336624914 list([9, 1, 7, 3, 8, 4]) 2 1]
[3.5164310278957407 list([9, 1, 7, 3, 5, 4]) 2 1]
[3.676385847190728 list([2, 1, 4, 3, 9, 8]) 2 1]
[3

[[(34, 36), (-53, -37), 1, (64.63745044476924, 49.51767361255979)], [(-53, -37), (22, 32), 1, (38.8329756778952, 64.63745044476924)], [(22, 32), (-25, -18), 1, (30.805843601498726, 38.8329756778952)], [(-25, -18), (8, 19), 1, (20.615528128088304, 30.805843601498726)], [(8, 19), (-1, -12), 1, (12.041594578792296, 20.615528128088304)]]
1000
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results/SSSM-000242.txt
[6.776693066882837 list([1, 4, 2, 7, 3, 5, 6, 9, 8]) 3 3]
[6.368062258680647 list([4, 1, 7, 2, 8, 3, 5, 6, 9]) 4 2]
[5.905551475234367 list([1, 4, 6, 7, 2, 8, 3, 9, 5]) 3 3]
[6.365331841166732 list([5, 8, 6, 7, 3, 4, 2, 9, 1]) 3 3]
[6.121659120341625 list([1, 4, 6, 8, 3, 7, 2, 9, 5]) 3 3]
[6.6729360315626005 list([1, 7, 2, 4, 3, 5, 6, 9, 8]) 3 3]
[6.6630697464377375 list([1, 4, 2, 7, 3, 8, 6, 9, 5]) 3 3]
[6.367657538339753 list([1, 7, 6, 8, 3, 4, 2, 9, 5]) 3 3]
[6.522209617486475 list([5, 1, 7, 2, 4, 3, 8,

...

951
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results/SSSM-B3-00053.txt
[3.8801329008643664 list([1, 2, 8, 7, 6, 9]) 2 1]
[3.8960185077118403 list([5, 3, 8, 7, 6, 9]) 2 1]
[3.7806446898235553 list([4, 2, 8, 7, 6, 9]) 2 1]
[3.9955067187526514 list([2, 3, 8, 7, 6, 9]) 2 1]
[3.8320455162968137 list([1, 2, 8, 7, 3, 6]) 2 1]
[3.6855989068900876 list([1, 2, 8, 7, 3, 5]) 2 1]
[3.7325573052560026 list([4, 2, 8, 7, 3, 6]) 2 1]
[3.5861106958492766 list([4, 2, 8, 7, 3, 5]) 2 1]
[3.5557044400799978 list([5, 3, 8, 7, 2, 6]) 2 1]
[3.5557044400799978 list([5, 3, 8, 7, 2, 4]) 2 1]
[3.8410203093887496 list([2, 3, 5, 4, 6, 9]) 2 1]
[3.9955067187526514 list([1, 3, 8, 7, 6, 9]) 2 1]
[3.801639260527535 list([1, 3, 8, 7, 2, 5]) 2 1]
[3.587239682050914 list([1, 2, 8, 4, 3, 6]) 2 1]
[3.492994809286839 list([5, 3, 8, 4, 6, 9]) 2 1]
[3.59248302032765 list([2, 3, 8, 4, 6, 9]) 2 1]
[3.8320455162968137 list([1, 2, 8, 7, 3, 9]) 2 1]
[

318
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results/SSSM-B3-00061.txt
[2.6502079620404335 list([4, 6, 3, 1, 7]) 1 1]
[2.8238040555092807 list([4, 9, 6, 1, 7]) 1 1]
[2.699235835728671 list([2, 9, 6, 1, 7]) 1 1]
[2.745182708871017 list([7, 9, 6, 4, 8]) 1 1]
[2.745182708871017 list([4, 6, 3, 1, 5]) 1 1]
[2.613126282843347 list([7, 6, 3, 1, 5]) 1 1]
[2.654649022770217 list([2, 9, 6, 4, 8]) 1 1]
[2.8817044768794244 list([7, 9, 6, 1, 8]) 1 1]
[2.9157390105592347 list([4, 9, 6, 1, 8]) 1 1]
[2.7911707907786245 list([2, 9, 6, 1, 8]) 1 1]
[2.742142917090387 list([4, 6, 3, 1, 8]) 1 1]
[2.7628575364732213 list([7, 9, 6, 2, 5]) 1 1]
[2.610086491062717 list([7, 6, 3, 1, 8]) 1 1]
[2.796892070153031 list([4, 9, 6, 2, 5]) 1 1]
[2.740516878437416 list([1, 9, 6, 4, 8]) 1 1]
[2.8847442686600537 list([7, 9, 6, 1, 5]) 1 1]
[2.918778802339864 list([4, 9, 6, 1, 5]) 1 1]
[2.7942105825592543 list([2, 9, 6, 1, 5]) 1 1]
[2.6455421

[[(-12, -17), (14, -17), 1, (22.02271554554524, 20.808652046684813)], [(14, -17), (-1, 17), 1, (17.029386365926403, 22.02271554554524)], [(-1, 17), (15, 15), 1, (21.213203435596427, 17.029386365926403)]]
908
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results/SSSM-B3-00068.txt
[3.829318360715841 list([3, 5, 8, 4, 2, 9]) 2 1]
[3.974698452815961 list([3, 5, 8, 4, 2, 7]) 2 1]
[3.682871751309115 list([6, 5, 8, 4, 2, 9]) 2 1]
[3.8282518434092347 list([6, 5, 8, 4, 2, 7]) 2 1]
[3.2537866378873463 list([3, 6, 9, 8, 5, 7]) 2 1]
[3.783435549647491 list([3, 5, 8, 4, 1, 7]) 2 1]
[3.277346179546564 list([3, 2, 5, 4, 1, 7]) 2 1]
[3.613429398581547 list([3, 6, 9, 5, 2, 4]) 2 1]
[3.6369889402407645 list([6, 5, 8, 4, 1, 7]) 2 1]
[3.6369889402407645 list([3, 6, 9, 5, 2, 8]) 2 1]
[3.6369889402407645 list([2, 5, 8, 4, 1, 7]) 2 1]
[3.1547367381353286 list([3, 2, 5, 4, 1, 8]) 2 1]
[3.050318293050718 list([3, 5, 9, 7, 1, 8]) 2 1]

603
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results/SSSM-B3-00078.txt
[2.3637539232770104 list([7, 8, 3, 4, 6]) 1 1]
[2.6268912404217546 list([5, 6, 1, 7, 9]) 1 1]
[2.9888820896174195 list([8, 9, 2, 7, 6]) 1 1]
[2.223612357155783 list([7, 8, 3, 4, 9]) 1 1]
[2.288217778259872 list([5, 6, 1, 8, 3]) 1 1]
[2.632565498430801 list([8, 9, 3, 4, 6]) 1 1]
[2.957007224746323 list([8, 9, 2, 7, 5]) 1 1]
[2.3758046702810836 list([7, 8, 3, 4, 2]) 1 1]
[2.3670320873790054 list([5, 9, 1, 8, 3]) 1 1]
[2.559061779941969 list([5, 6, 1, 7, 2]) 1 1]
[2.446912294862864 list([5, 9, 1, 8, 6]) 1 1]
[2.799401255835052 list([8, 9, 3, 5, 6]) 1 1]
[2.799401255835052 list([7, 8, 2, 4, 5]) 1 1]
[2.6446162454348743 list([8, 9, 3, 4, 2]) 1 1]
[2.8072697370579016 list([8, 9, 1, 7, 5]) 1 1]
[2.7912490534056293 list([5, 9, 2, 8, 6]) 1 1]
[2.7912490534056293 list([4, 8, 1, 7, 5]) 1 1]
[2.5494285469128877 list([2, 6, 1, 7, 5]) 1 1]
[2.77919

[[(20, -3), (-32, 25), 1, (40.607881008493905, 20.223748416156685)], [(-32, 25), (36, 12), 1, (37.94733192202055, 40.607881008493905)], [(36, 12), (-18, -16), 1, (24.08318915758459, 37.94733192202055)]]
1000
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results
/home/user/zy/attack-on-pattern-pin/results/no-end/no-end_results/SSSM-B3-00087.txt
[4.942047755805365 list([5, 3, 4, 9, 8, 6, 2]) 2 2]
[4.636354662807442 list([5, 3, 1, 8, 7, 9, 4]) 2 2]
[4.636354662807442 list([5, 3, 1, 8, 7, 9, 2]) 2 2]
[4.528962487034513 list([4, 5, 7, 9, 8, 6, 2]) 2 2]
[4.469399395634904 list([6, 3, 1, 8, 7, 9, 4]) 2 2]
[4.469399395634904 list([6, 3, 1, 8, 7, 9, 2]) 2 2]
[4.510922135561773 list([2, 3, 1, 8, 7, 9, 4]) 2 2]
[4.683323706392846 list([5, 3, 1, 8, 7, 6, 4]) 2 2]
[4.816615228559696 list([1, 3, 4, 9, 8, 6, 2]) 2 2]
[4.213570493631499 list([4, 2, 7, 9, 8, 3, 1]) 2 2]
[4.148557360265825 list([5, 6, 1, 8, 7, 9, 3]) 2 2]
[4.320627995060088 list([4, 2, 7, 9, 8, 6, 5]) 2 2]
[4.320627995060088